In [1]:
import rospy
import numpy as np
from turtlesim.srv import Spawn
from turtlesim.msg import Pose
from geometry_msgs.msg import Twist
from pkg_tsr.srv import Task
import math

rospy.init_node("serwis_node_test")

In [2]:
class Robot():
    position = Pose()
    def __init__(self, name, x, y):
        self.name = name
        self.follow_service = rospy.Service(name+'/go_to_goal', Task, self.go_to_goal_spiral)
        self.velocity_pub = rospy.Publisher(name+'/cmd_vel', Twist, queue_size=10)
        self.pose_sub = rospy.Subscriber(name+'/pose',Pose, self.callback_pose)
        
        create_new_turtle = rospy.ServiceProxy('spawn', Spawn)
        create_new_turtle(x, y, 0, name)
        
    def go_to_goal(self, msg_srv):
        max_speed = 0.3 # dla liniowych i obrotowych
        tolerance = 0.05 # max odleglosc od punktu
        
        # obracanie się w stronę punktu
        while True:
            celTheta = math.atan2(msg_srv.y - self.position.y,msg_srv.x - self.position.x)
            celDelta = celTheta - self.position.theta
            if abs(celDelta) < tolerance*0.005:
                break
            # poprawiamy
            msg_vel = Twist()
            speed = np.clip(0.2*(abs(celDelta))+0.01,0,max_speed)
            if celDelta > 0:
                msg_vel.angular.z = speed
            else:
                msg_vel.angular.z = -speed
            self.velocity_pub.publish(msg_vel)
            
        # jazda do punktu
        while True:
            celDelta = math.sqrt((msg_srv.x - self.position.x)**2 + (msg_srv.y - self.position.y)**2)
            if celDelta < tolerance:
                break
            #poprawiamy
            msg_vel = Twist()
            msg_vel.linear.x = np.clip(0.25*celDelta+0.025,0,max_speed)
            self.velocity_pub.publish(msg_vel)
            
        return "finished"
    def go_to_goal_spiral(self, msg_srv):
        self.go_to_goal(msg_srv)
        start = rospy.get_time()
        while True:
            time = rospy.get_time()-start
            if time > 20:
                break
            msg = Twist()
            msg.linear.x = 0.3
            msg.angular.z = 3/(rospy.get_time()-start+1)
            self.velocity_pub.publish(msg)
            rospy.sleep(1)
        return "finished"    
        
    def callback_pose(self, data):
        self.position = data
    
    def unregister(self):
        self.follow_service.shutdown()
        
# funkcja pomocnicza do kierowania robotem
def kieruj(robot,listaPunktow):
    klient = rospy.ServiceProxy(robot.name+'/go_to_goal',Task)
    for punkt in listaPunktow:
        klient(punkt.x,punkt.y)
        
class Punkt():
    x = 0.0
    y = 0.0
    def __init__(self,x,y):
        self.x = x
        self.y = y

In [3]:
!rosservice call reset

In [24]:
robocik.unregister()

NameError: name 'robocik' is not defined

In [4]:
# UZUPEŁNIĆ - utworzyć klienta i zlecić zadanie dojazdu do kilku różnych punktów
robocik = Robot("Zen55",4,1)
droga = [ 
    Punkt(5,5),
    Punkt(5,8)
]
kieruj(robocik,droga)

In [5]:
robocik.position

x: 5.103328704833984
y: 6.0348310470581055
theta: -0.2393801212310791
linear_velocity: 0.0
angular_velocity: 0.0